#### Llama2 7B torch.float16 추론 
> * Llama2 모델은 GPU T4x16G 에서는 사용할 수 없다. 공식문서에서는 T4x27G 이상 사용을 권하고 있다.
```
1. casllm-base-7b-hf: Meta Llama2 7B
2. parameter data type: touch.float16
3. qutanization: not quantized
4. processor: cpu
5. write type: pytorch bin
6. transformers: AutoModelForCausalLM, pipeline
```

In [1]:
import torch
from transformers import AutoModelForCausalLM, pipeline

In [2]:
# GPU 검사
if torch.cuda.is_available():
    print("CUDA is available. GPU can be used.")
else:
    print("CUDA is not available. Using CPU instead.")
    
    
if torch.cuda.is_available():
    torch.cuda.empty_cache() # 사용되지 않는 캐시된 메모리 해제
    print("GPU cache is emptied.")

CUDA is available. GPU can be used.
GPU cache is emptied.


In [3]:
# KoAlpaca-ployglot 12.8B (Non 양자화 모델)
# 파일형식: Hugging Transformers 의 Safe serialization 모델
# 파일크기: 1G X 28개
model_id = "/data/aibiseo/models/casllm-base-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"":0}, torch_dtype=torch.bfloat16,)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# 모델이 올라온 위치
# 로딩 모델 위치
next(model.parameters()).device

device(type='cuda', index=0)

In [5]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

First parameter data type: torch.bfloat16


In [ ]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

In [6]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [7]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [8]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [9]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템 감리 절차 과정에 대해 설명해줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

### 질문:
정보화시스템 감리 절차 과정에 대해 설명해줘


In [10]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.58 GiB of which 14.38 MiB is free. Including non-PyTorch memory, this process has 14.56 GiB memory in use. Of the allocated memory 13.80 GiB is allocated by PyTorch, and 56.41 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)